In [583]:
import numpy as np
import matplotlib.pyplot as plt

In [965]:
class NeuralNet:
    def __init__(self,nodes,indims,activations,
                  actders, cost, task='Classifcation'):
        self.nodes=nodes
        self.layers=len(nodes)
        self.indims=indims
        self.activations=activations
        self.actders=actders
        self.task=task
        self.cost=cost
        self.weights={}
        self.biases={}
    
    def weightinit(self):
        self.weights['W0']=np.random.randn(self.indims,self.nodes[0])
        self.biases['B0']=np.random.randn(1,self.nodes[0])
        for i in range(1,self.layers):
            self.weights['W'+str(i)]=np.random.randn(self.nodes[i-1],self.nodes[i])
            self.biases['B'+str(i)]=np.random.randn(1,self.nodes[i])
    
    def get_one_hot(self,targets):
        targets=np.asarray(targets).reshape(-1)
        num_classes=len(np.unique(targets))
        indices=np.searchsorted(np.unique(targets),targets)
        res = np.eye(num_classes)[indices]
        return res.reshape(len(targets),num_classes)

    def mat_ohe(self,data):
        ohe_cols=np.apply_along_axis(self.get_one_hot,0,data[:,[0]])[:,:,0]
        data=np.hstack((data,ohe_cols))
        data=np.delete(data,cols_to_ohe,1)
        return data.astype(float)
            
    def predict(self, X):
        self.Z={}
        self.Zprime={}
        self.Z['Z0']=X
        if not self.weights:
            self.weightinit()
        for i in range(self.layers-1):
            self.Z['Z'+str(i+1)]=self.activations[i](self.Z['Z'+str(i)]@self.weights['W'+str(i)]\
                                                    +self.biases["B"+str(i)])
            self.Zprime['Zprime'+str(i)]=self.actders[i](self.Z['Z'+str(i)]@self.weights['W'+str(i)]\
                                                        +self.biases["B"+str(i)])
        if self.task=='Classification':
            #Convert to array in case a matrix was passed
            self.probabilities=np.array(self.activations[self.layers-1](self.Z['Z'+str(self.layers-1)]\
                                                                    @self.weights['W'+str(self.layers-1)]\
                                                                    +self.biases["B"+str(self.layers-1)]))
            self.predictions=np.eye(self.probabilities.shape[1])[np.argmax(self.probabilities, axis=1)]
        elif self.task=='Regression':
            self.predictions=self.Z['Z'+str(self.layers-1)]@self.weights['W'+str(self.layers-1)]\
                                     +self.biases["B"+str(self.layers-1)]
        else:
            self.predictions="Invalid task. Must use regression or classification.Default is classification"                                                              
            
    def train(self,X,y,epochs,eta,X_val=False,y_val=False):
        self.valerror=[]
        self.error=[]
        d={}
        self.predict(X)
        
        if (self.task=="Classification" and y.shape[1]<self.nodes[-1]):
            y=self.mat_ohe(y)
        
        for i in range(epochs):
            if self.task=='Classification':
                d['d'+str(self.layers-1)]=self.probabilities-y
            elif self.task=='Regression':
                d['d'+str(self.layers-1)]=self.predictions-y
            else:
                return self.predictions
            
            for j in range(self.layers-1):
                d['d'+str(self.layers-2-j)]=np.multiply(d['d'+str(self.layers-1-j)]\
                                                        @self.weights['W'+str(self.layers-1-j)].T,\
                                                        self.Zprime['Zprime'+str(self.layers-2-j)])

            for j in range(self.layers):
                self.weights['W'+str(j)]=self.weights['W'+str(j)]-eta*self.Z['Z'+str(j)].T@d['d'+str(j)]
                self.biases['B'+str(j)]=self.biases['B'+str(j)]-eta*np.sum(d['d'+str(j)])

            if self.task=='Classification':
                if y_val:
                    self.predict(X_val)
                    self.valerror.append(self.cost(y_val,self.probabilities))
                self.predict(X)
                self.error.append(self.cost(y,self.probabilities))
            else:
                if y_val:
                    self.predict(X_val)
                    self.valerror.append(self.cost(y_val,self.predictions))
                self.predict(X)
                self.error.append(self.cost(y,self.predictions))